![](pictures/shopping_propensity.png)

<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
Ноутбук от 30.09.23 
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>   
**_"Data Science Analyst"_  
_МОДЕЛЬ СКЛОННОСТИ КЛИЕНТОВ К ПОКУПКАМ В ФЕДЕРАЛЬНОЙ СЕТИ ПРОДАЖ СПОРТИВНЫХ ТОВАРОВ_**  

Выполнено по технологии CRISP-DM:  
Аргентов Сергей, г.Липецк  
https://argentov.pro
<b>  </b>  
<b>  </b>  

<h1>СОДЕРЖАНИЕ<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#2.-Обогащение-данных-таблицы-personal_data" data-toc-modified-id="2.-Обогащение-данных-таблицы-personal_data-1">2. Обогащение данных таблицы personal_data</a></span><ul class="toc-item"><li><span><a href="#2.1.-Получим-данные-из-дополнительного-источника" data-toc-modified-id="2.1.-Получим-данные-из-дополнительного-источника-1.1">2.1. Получим данные из дополнительного источника</a></span></li><li><span><a href="#2.2.-Восстановим-данные-моделями-машинного-обучения" data-toc-modified-id="2.2.-Восстановим-данные-моделями-машинного-обучения-1.2">2.2. Восстановим данные моделями машинного обучения</a></span></li></ul></li></ul></div>

In [1]:
# Загрузка биьлиотек
import pandas as pd
import sqlite3 as db

import datetime as dt
import multiprocessing
from multiprocessing import Pool, Manager

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import pprint
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score

# 2. Обогащение данных таблицы personal_data

In [2]:
# Загрузим данные из базы данных (с добавленными ранее восстановленными данными по product_sex и colour)
with db.connect('data/result_data/shop_database_.db') as con:
    rows = con.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    dfs = {}
    for row in rows:
        dfs[row[0]] = pd.read_sql(f'SELECT * FROM {row[0]}', con)
        print(f'\n{row[0]}')
        print(dfs[row[0]].info())


personal_data_coeffs
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104989 entries, 0 to 104988
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             104989 non-null  int64  
 1   lbt_coef       104989 non-null  float64
 2   ac_coef        104989 non-null  float64
 3   sm_coef        104989 non-null  float64
 4   personal_coef  104989 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 4.0 MB
None

personal_data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89241 entries, 0 to 89240
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         89241 non-null  int64 
 1   gender     89241 non-null  int64 
 2   age        89241 non-null  int64 
 3   education  89241 non-null  object
 4   city       89241 non-null  int64 
 5   country    89241 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 4.1+ MB
None

purchas

## 2.1. Получим данные из дополнительного источника

In [3]:
# Прочитаем восстановленные данные
df_recovered  = pd.read_csv('data/result_data/personal_data.csv')
df_recovered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15748 entries, 0 to 15747
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         15748 non-null  int64 
 1   age        15748 non-null  int64 
 2   education  15748 non-null  object
 3   city       15748 non-null  int64 
 4   country    15748 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 615.3+ KB


In [4]:
# Добавим восстановленные данные в общий датафрейм
dfs['personal_data_recovered'] = pd.concat([dfs['personal_data'], df_recovered], axis=0)
dfs['personal_data_recovered'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104989 entries, 0 to 15747
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         104989 non-null  int64  
 1   gender     89241 non-null   float64
 2   age        104989 non-null  int64  
 3   education  104989 non-null  object 
 4   city       104989 non-null  int64  
 5   country    104989 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 5.6+ MB


## 2.2. Восстановим данные моделями машинного обучения

Необхлодимо выбрать и обучить модель для прогнозирования признака "gender" в записях с недостающими значениями такого признака.  
Для обучения подготовим датасет

In [5]:
# С целью получения максимально большого массива данных для обучении модели
# объединим датасеты 'purchases', 'personal_data_coeffs', 'personal_data'
df_personal = dfs['personal_data_recovered'].set_index('id').\
                                             join(dfs['personal_data_coeffs'].set_index('id'))
df = dfs['purchases'].set_index('id').join(df_personal)
# Перенесём id покупателя в отдельный признак итогового датасета и сбросим индексы
# (это необходимо для дальнейшей работы при выявлении аномальных записей в датасете)
df.reset_index(drop=False, inplace=True)

In [6]:
# Выделим данные с заполненным признаком 'gender' для формирования датасета для обучения модели классификации
df_train_test = df[~df.gender.isna()]
df_train_test.reset_index(inplace=True)
df_train_test.head(3)

,index,id,product,colour,cost,product_sex,base_sale,dt,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef
0,0,0,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
1,1,0,Стол Outventure,зеленый,1499,2.0,0,37,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
2,2,0,Набор Outventure: стол + 4 стула,бежевый,4799,2.0,0,37,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072


Перед обучением модели целесообразно проверить данные на аномалии и исключить такие данные из обучающего датасета.  
1) При проверки датасета на аномальные записи используем три метода, результаты которых дополним друг другом для повышения качества обучающей выборки.  
2) При проверки данных на аномалии не будем учитывать значения признаков `id`, `lbt_coef`, `ac_coef`, `sm_coef` - так как эти данные уникальны почти для каждого покупателя, а значит не принесут пользы для факторной оценки данных и дальнейшего обучения модели.  
3) При использовании методов попризнакового выявления аномалий не будем учитывать признаки `product_sex`,`base_sale`,`education` - так как они имеют понятные уникальные значения без аномалий (выявлено визуально). Также не будем учитывать признак `gender` - так как это целевая переменная для обуения модели.  

In [7]:
# Определим колонки для попризнакового выявления аномалий
check_columns = ['index', 'product', 'colour', 'cost', 'dt', 'age', 'city', 'country', 'personal_coef']

In [8]:
# Проведём кодирование каждого признака 
df_fit = pd.DataFrame()
for column in check_columns:
    le = LabelEncoder()
    df_fit[column] = le.fit_transform(df_train_test[column])
df_fit.head()

,index,product,colour,cost,dt,age,city,country,personal_coef
0,0,3259,203,1475,7,31,694,16,13
1,1,17773,433,371,37,31,694,16,13
2,2,12187,35,882,37,31,694,16,13
3,3,15582,1456,1256,0,30,627,16,13
4,4,21057,87,907,0,30,627,16,13


In [9]:
df_fit.index = df_fit['index']
df_fit.drop(columns=['index'], axis=1, inplace=True)

In [10]:
# Инициализируем словарь индексов, в котором будем фиксировать:
# метод выявления аномалий; список индексов записей-аномалий
indexs = {'Quantile': []}

In [11]:
# Заполним словарь индексов записей-аномалий, индексами аномалий,
# выявленными квантилями более 3-сигма
for column in df_fit.columns:
    quantile_indexs = list(df_fit[df_fit[column] > df_fit[column].quantile(0.997)].index)
    if len(quantile_indexs) > 0:
        indexs['Quantile'].extend(quantile_indexs)
indexs['Quantile'] = list(set(indexs['Quantile']))

In [12]:
# Заполним словарь индексов записей-аномалий, индексами аномалий,
# выявленными Локалфактором и Изолесом
lof = LocalOutlierFactor(n_neighbors=693, n_jobs=-1)
ifo = IsolationForest(max_samples=12300, n_jobs=-1, random_state=42)
pred_lof = lof.fit_predict(df_fit)
pred_ifo = ifo.fit_predict(df_fit)
indexs['LocalOutlierFactor'] = list(df_fit[pred_lof == -1].index)
indexs['IsolationForest'] = list(df_fit[pred_ifo == -1].index)

In [13]:
# Сформируемый единый список индексов, полученных всеми методами
indexs_all = []
# Определим долю выбросов, выявленную каждым методом
for method in indexs.keys():
    count_anomaly = len(indexs[method])
    print(f"Методом '{method}' выявлено {count_anomaly} записей-аномалий,", end=' ')
    print(f"что составляет {round(count_anomaly/len(df_train_test) * 100, 2)}% всех записей")
    indexs_all.extend(indexs[method])
indexs_all = list(set(indexs_all))
indexs_all.sort()
print(f'\nИТОГО вывленных записей-аномалий {len(indexs_all)},', end=' ')
print(f'что составляет {round(len(indexs_all)/len(df_train_test) * 100, 2)}% всех записей')

Методом 'Quantile' выявлено 8928 записей-аномалий, что составляет 1.33% всех записей
Методом 'LocalOutlierFactor' выявлено 35630 записей-аномалий, что составляет 5.32% всех записей
Методом 'IsolationForest' выявлено 61030 записей-аномалий, что составляет 9.11% всех записей

ИТОГО вывленных записей-аномалий 82150, что составляет 12.26% всех записей


In [14]:
# Удалим из обучающего датасета записи с признаками аномалий
df_train_test_norm = df_train_test.drop(index=indexs_all)

In [15]:
# Проверим сбалансированность записей датасета по целевому признаку
balance = df_train_test_norm.gender.value_counts()
print(f'Положительный класс: {balance[1]} ({round(balance[1]/(balance[0]+balance[1])*100)})%')
print(f'Отрицательный класс: {balance[0]} ({round(balance[0]/(balance[0]+balance[1])*100)})%')

Положительный класс: 324347 (55)%
Отрицательный класс: 263459 (45)%


**Вывод:** Так как датасет неполностью сбалансирован используем при обучении моделей параметр `class_weight='balanced'`, который позволяет учесть разбалансировку до соотношения (57%/43%)

In [16]:
df_train_test_norm.head(3)

,index,id,product,colour,cost,product_sex,base_sale,dt,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef
0,0,0,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
1,1,0,Стол Outventure,зеленый,1499,2.0,0,37,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
2,2,0,Набор Outventure: стол + 4 стула,бежевый,4799,2.0,0,37,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072


In [17]:
# Закодируем категориальные признаки 'product', 'colour', 'education':
le1 = LabelEncoder()
df_train_test_norm_ = df_train_test_norm.copy()
for col in ['product', 'colour', 'education']:
    le1.fit(list(set(df_train_test_norm_[col].dropna().unique())))
    df_train_test_norm_[col] = le1.transform(df_train_test_norm_[col])
df_train_test_norm_.head(3)

,index,id,product,colour,cost,product_sex,base_sale,dt,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef
0,0,0,2753,179,13599,0.0,1,7,0.0,36,1,1201,32,5.078678,-0.307147,0.959027,0.5072
1,1,0,16189,400,1499,2.0,0,37,0.0,36,1,1201,32,5.078678,-0.307147,0.959027,0.5072
2,2,0,11116,21,4799,2.0,0,37,0.0,36,1,1201,32,5.078678,-0.307147,0.959027,0.5072


In [18]:
# Стандартизуем не бинарные признаки, которые будем использовать при обучении
ss = StandardScaler()
for col in ['product', 'colour', 'cost', 'dt', 'age', 'city', 'country', 'personal_coef']:
    df_train_test_norm_[col] = ss.fit_transform(df_train_test_norm_[[col]])
df_train_test_norm_.head(3)

,index,id,product,colour,cost,product_sex,base_sale,dt,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef
0,0,0,-1.358996,-1.533852,1.542745,0.0,1,-1.046609,0.0,-0.27796,1,0.828864,0.0,5.078678,-0.307147,0.959027,0.803887
1,1,0,0.979997,-1.034691,-0.530135,2.0,0,0.705950,0.0,-0.27796,1,0.828864,0.0,5.078678,-0.307147,0.959027,0.803887
2,2,0,0.096869,-1.890718,0.035196,2.0,0,0.705950,0.0,-0.27796,1,0.828864,0.0,5.078678,-0.307147,0.959027,0.803887


In [19]:
# Признаки используемые при обучении модели
cols = ['product', 'colour', 'cost', 'product_sex', 'base_sale', 'dt', 'age', 'education',
        'city', 'country', 'personal_coef']

In [20]:
# Обучим модель "Случайный лес Деревьев принятия решения" на всех имеющихся данных 
rf = RandomForestClassifier(class_weight='balanced')
rf.fit(df_train_test_norm_[cols], df_train_test_norm_['gender'])

RandomForestClassifier(class_weight='balanced')

In [21]:
# Выберем, закодируем и стандартизуем признаки записей,
# в которых необходимо восстановить знпачени признака 'gender'
df_pred = df[df.gender.isna()]
gender_index = df_pred.index

# Закодируем категориальные признаки 'product', 'colour', 'education':
le2 = LabelEncoder()
df_pred_ = df_pred.copy()
for col in ['product', 'colour', 'education']:
    le1.fit(list(set(df_pred_[col].dropna().unique())))
    df_pred_[col] = le1.transform(df_pred_[col])

# Стандартизуем не бинарные признаки, которые будем использовать при обучении
ss = StandardScaler()
for col in ['product', 'colour', 'cost', 'dt', 'age', 'city', 'country', 'personal_coef']:
    df_pred_[col] = ss.fit_transform(df_pred_[[col]])
df_pred_.head(3)

,id,product,colour,cost,product_sex,base_sale,dt,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef
3,3,-1.404714,-1.711752,-0.370080,1.0,0,-0.693454,NaN,-0.601036,1,0.018259,0.003842,7.764766,-0.030225,0.79472,-0.474307
4,3,0.091138,-0.553376,0.177158,2.0,0,0.116252,NaN,-0.601036,1,0.018259,0.003842,7.764766,-0.030225,0.79472,-0.474307
5,3,-0.859274,-1.711752,-0.252814,1.0,1,0.521105,NaN,-0.601036,1,0.018259,0.003842,7.764766,-0.030225,0.79472,-0.474307


In [22]:
# Восстановим признак 'gender' с помощью обученной ранее модели, используя признаки с непустыми данными
gender_pred = rf.predict(df_pred_[cols])
df_gender = pd.DataFrame({'gender': gender_pred})
df_gender.index = gender_index

In [23]:
# Добавим восстановленные значения в общий датафрейм
for ind in df_gender.index:
    df.loc[ind, 'gender'] = df_gender.loc[ind, 'gender']
df.head(6)

,id,product,colour,cost,product_sex,base_sale,dt,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef
0,0,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
1,0,Стол Outventure,зеленый,1499,2.0,0,37,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
2,0,Набор Outventure: стол + 4 стула,бежевый,4799,2.0,0,37,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072
3,3,Бутсы мужские GSD Astro,белый,1599,1.0,0,13,1.0,31,среднее,1134,32,7.764766,-0.030225,0.794720,0.4304
4,3,Мяч футбольный PUMA TEAMFINAL 21.2 FIFA QUALIT...,многоцветный,7199,2.0,0,27,1.0,31,среднее,1134,32,7.764766,-0.030225,0.794720,0.4304
5,3,Кеды мужские FILA A-Low,белый,2799,1.0,1,34,1.0,31,среднее,1134,32,7.764766,-0.030225,0.794720,0.4304


In [24]:
# Загрузим итоговый датаcет в хранилище (база данных sqlite3)
connection = db.connect('data/purchases.db')
print("База данных создана и подключена")
df.to_sql("full_table", connection, if_exists="replace", index=False)
print("Таблица заполнена")
connection.close()
print("Соединение с базой данных закрыто")

База данных создана и подключена
Таблица заполнена
Соединение с базой данных закрыто


In [25]:
# Проверим полноту данных в итоговом датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786260 entries, 0 to 786259
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             786260 non-null  int64  
 1   product        786260 non-null  object 
 2   colour         786260 non-null  object 
 3   cost           786260 non-null  int64  
 4   product_sex    786260 non-null  float64
 5   base_sale      786260 non-null  int64  
 6   dt             786260 non-null  int64  
 7   gender         786260 non-null  float64
 8   age            786260 non-null  int64  
 9   education      786260 non-null  object 
 10  city           786260 non-null  int64  
 11  country        786260 non-null  int64  
 12  lbt_coef       786260 non-null  float64
 13  ac_coef        786260 non-null  float64
 14  sm_coef        786260 non-null  float64
 15  personal_coef  786260 non-null  float64
dtypes: float64(6), int64(7), object(3)
memory usage: 96.0+ MB


In [27]:
# Оптимизируем используемую память
# Определяем список загруженных датафреймов
dfs = []
local_variable_scope =locals().keys()
for elem in local_variable_scope:
    if 'df' in elem:
        dfs.append(elem)
# Удаляем не используемые датафреймы
for elem in dfs:
    del locals()[elem]